In [2]:
import numpy as np
import pickle
import cv2
import re

# -----------------------------------------
# 1) Load label file (ids, labels)
# -----------------------------------------
label_path = "data/NTU-RGB-D/x-view/small_test_label.pkl"
ids, labels = pickle.load(open(label_path, "rb"))

# labels 안에는 파일명(strings)
# 예: "S001C001P001R001A011.skeleton"

def extract_action_id(name):
    m = re.search(r"A(\d{3})", name)
    return int(m.group(1)) if m else -1

action_ids = np.array([extract_action_id(n) for n in ids])

print("라벨 종류:", sorted(set(action_ids)))

라벨 종류: [np.int64(1), np.int64(2), np.int64(8), np.int64(9), np.int64(11), np.int64(12), np.int64(13), np.int64(28), np.int64(29), np.int64(30), np.int64(33), np.int64(41)]


In [24]:
import numpy as np
import cv2
import subprocess

# -----------------------------------------
# Select target label
# -----------------------------------------
target_label = 12
indices = np.where(action_ids == target_label)[0]

if len(indices) == 0:
    raise ValueError(f"No sample with label {target_label}")

sample_idx = indices[0]
print("Selected sample:", ids[sample_idx])

# -----------------------------------------
# Load ST-GCN processed data
# -----------------------------------------
data_path = "data/NTU-RGB-D/x-view/small_test_data.npy"
data = np.load(data_path)  # (N,3,T,25,2)

sample = data[sample_idx]
x = sample[0]     # (T,25,2)
y = sample[1]
T = x.shape[0]
# -----------------------------------------
# Joint groups by functional role
# -----------------------------------------

groups = {
    "head":  [3,4],
    "body":  [1,2,21],
    "arms":  [5,6,7,8, 9,10,11,12],
    "hands": [22,23,24,25],
    "legs":  [13,14,15,16, 17,18,19,20],
    "feet":  [16,20]
}

colors = {
    "head":  (0,255,255),
    "body":  (255,120,0),
    "arms":  (0,0,255),
    "hands": (0,165,255),
    "legs":  (0,255,0),
    "feet":  (255,0,255)
}

# -----------------------------------------
# Bone connections
# -----------------------------------------
ntu_pairs = [
    (1,2),(2,21),(3,21),(4,3),(5,21),(6,5),(7,6),(8,7),
    (9,21),(10,9),(11,10),(12,11),(13,1),(14,13),(15,14),
    (16,15),(17,1),(18,17),(19,18),(20,19),
    (22,23),(23,8),(24,25),(25,12)
]

# -----------------------------------------
# Visualization setup
# -----------------------------------------
size = 900
fps = 15


temp_avi =  f"A{target_label:03d}_video.avi" # 임시파일
output_mp4 = f"A{target_label:03d}_video.mp4"

video = cv2.VideoWriter(temp_avi, cv2.VideoWriter_fourcc(*'XVID'), fps, (size,size))

def draw_grid(img, step=60):
    for i in range(0,size,step):
        cv2.line(img, (i,0),(i,size),(220,220,220),1)
        cv2.line(img, (0,i),(size,i),(220,220,220),1)

def normalize(coords):
    x, y = coords[:,0], coords[:,1]
    x = (x - x.min()) / (x.max() - x.min() + 1e-9) * (size - 200) + 100
    y = (y - y.min()) / (y.max() - y.min() + 1e-9) * (size - 200) + 100
    return np.vstack([x,y]).T

# -----------------------------------------
# Generate animation
# -----------------------------------------
for t in range(T):

    pts = np.zeros((25,2))
    for j in range(25):
        pts[j,0] = x[t,j,0]
        pts[j,1] = y[t,j,0]

    if np.all(pts == 0):   # skip empty frame
        continue

    pts = normalize(pts)
    pts[:,1] = size - pts[:,1]

    frame = np.ones((size,size,3), np.uint8) * 255
    draw_grid(frame)

    # Draw bones
    for a,b in ntu_pairs:
        a -= 1; b -= 1
        p1 = (int(pts[a,0]), int(pts[a,1]))
        p2 = (int(pts[b,0]), int(pts[b,1]))
        cv2.line(frame, p1, p2, (120,120,120), 3)

    # Draw joints + labels
    for group_name, joint_list in groups.items():
        color = colors[group_name]

        for j in joint_list:
            j -= 1
            p = (int(pts[j,0]), int(pts[j,1]))
            cv2.circle(frame, p, 12, color, -1)
            cv2.circle(frame, p, 14, (0,0,0), 2)
            cv2.putText(frame, str(j+1), (p[0]+10,p[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)

    cv2.putText(frame, f"Label {target_label} | Frame {t+1}",
                (20,60), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,0,0), 3)

    video.write(frame)

video.release()

print("AVI Saved:", temp_avi)

# -----------------------------------------
# Convert AVI → H.264 MP4 (브라우저 100% 호환)
# -----------------------------------------
cmd = [
    "ffmpeg",
    "-y",
    "-i", temp_avi,
    "-vcodec", "libx264",
    "-preset", "slow",
    "-crf", "18",
    "-pix_fmt", "yuv420p",
    output_mp4
]

sample = data[sample_idx]   # shape (3,T,25,2)
np.save(f"A{target_label:03d}_video.npy", sample)
print("MP4 Saved:", output_mp4)


Selected sample: S009C001P015R001A012.skeleton
AVI Saved: A012_video.avi
MP4 Saved: A012_video.mp4
